In [2]:
import pandas as pd
import numpy as np
import os
import zipfile
import csv

In [52]:
df = pd.read_csv('metaData.csv', na_values='-')
df.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15


In [4]:
df.groupby('busRoute').size()

busRoute
31     17
32     26
33    155
46    123
72    158
83    846
N1     16
N2     39
N4     18
dtype: int64

In [5]:
df.dtypes

name                             object
busNumber                         int64
startTime_iso                    object
startTime_unix                    int64
endTime_iso                      object
endTime_unix                      int64
drivenDistance                  float64
busRoute                         object
energyConsumption               float64
itcs_numberOfPassengers_mean    float64
itcs_numberOfPassengers_min       int64
itcs_numberOfPassengers_max       int64
status_gridIsAvailable_mean     float64
temperature_ambient_mean        float64
temperature_ambient_min         float64
temperature_ambient_max         float64
dtype: object

In [6]:
df.tail()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500
1408,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10T23:34:46Z,1670715286,2022-12-11T03:29:05Z,1670729345,72293.83,N2,5.350956e+08,15.99320,0,52,0.459492,276.0036,273.15,291.1500


### 1. Extract all trips with `busRoute` 83

In [7]:
df[df['busRoute']=='83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


### 2. Extract all trips where `busRoute` is not a number

In [104]:
df[df['busRoute'].str.contains(r'\D', na=False)]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
533,B183_2021-12-18_23-37-00_2021-12-19_03-38-35,183,2021-12-18T23:37:00Z,1639870620,2021-12-19T03:38:35Z,1639885115,76216.06,N4,481350300.0,9.198582,0,37,0.491653,276.8632,275.15,288.15
553,B183_2022-01-07_23-40-43_2022-01-08_03-31-21,183,2022-01-07T23:40:43Z,1641598843,2022-01-08T03:31:21Z,1641612681,68557.06,N2,453625100.0,4.626984,0,13,0.427488,276.9673,275.15,287.15
554,B183_2022-01-08_23-40-17_2022-01-09_03-35-32,183,2022-01-08T23:40:17Z,1641685217,2022-01-09T03:35:32Z,1641699332,67962.92,N2,475383300.0,7.495495,0,26,0.515514,278.5645,277.15,288.15
561,B183_2022-01-15_23-41-46_2022-01-16_03-40-23,183,2022-01-15T23:41:46Z,1642290106,2022-01-16T03:40:23Z,1642304423,77156.70,N1,525168300.0,6.512500,0,32,0.473809,274.9937,273.15,286.15
568,B183_2022-01-21_23-35-40_2022-01-22_03-26-24,183,2022-01-21T23:35:40Z,1642808140,2022-01-22T03:26:24Z,1642821984,71917.75,N2,455476000.0,5.357143,0,23,0.493608,275.3073,274.15,281.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,434776600.0,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,399773700.0,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,447553400.0,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,451916500.0,20.105260,0,74,0.495739,279.4540,277.15,291.15


### 3. For each (busNumber, busRoute) pair, determine the number of trips

In [13]:
df.groupby(['busRoute', 'busNumber'], as_index=False).size()

,busRoute,busNumber,size
0,31,183,12
1,31,208,5
2,32,183,12
3,32,208,14
4,33,183,130
5,33,208,25
6,46,183,104
7,46,208,19
8,72,183,114
9,72,208,44


### 4. For each trip, compute the ratio between the energy consumption and the average number of passengers

In [8]:
df['ratio'] = df['energyConsumption']/df['itcs_numberOfPassengers_mean']

In [15]:
df.head()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,NaN,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,NaN,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15,2.613190e+07


In [16]:
df.tail()

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,ratio
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.80899,0,83,0.739349,279.6404,278.15,291.1500,1.070215e+07
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655,5.338462e+07
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.87940,0,102,0.439916,279.1724,277.15,292.1500,4.738047e+07
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.10526,0,74,0.495739,279.4540,277.15,291.1500,2.247753e+07
1408,B208_2022-12-10_23-34-46_2022-12-11_03-29-05,208,2022-12-10T23:34:46Z,1670715286,2022-12-11T03:29:05Z,1670729345,72293.83,N2,5.350956e+08,15.99320,0,52,0.459492,276.0036,273.15,291.1500,3.345769e+07


### 5. For each station (`itcs_stopName`), determine the average number of passengers.

In [14]:
df_stops= pd.DataFrame()
for name in df['name']:
    path= f'/Users/andreacardinali/Documents/ZTBus_samples/{name}.csv'
    if os.path.exists(path):
        file= pd.read_csv(path, na_values='-', usecols=['itcs_numberOfPassengers', 'itcs_stopName']).dropna(subset=['itcs_stopName'])
        df_stops = pd.concat([df_stops, file])

In [15]:
df_stops.groupby('itcs_stopName', as_index=False)['itcs_numberOfPassengers'].mean().round(2)

,itcs_stopName,itcs_numberOfPassengers
0,"Zürich, Albisrank",17.79
1,"Zürich, Albisriederplatz",26.95
2,"Zürich, Altes Krematorium",22.20
3,"Zürich, Bahnhof Altstetten",10.30
4,"Zürich, Bahnhof Hardbrücke",28.25
...,...,...
71,"Zürich, Vogelsangstrasse",15.90
72,"Zürich, Waffenplatz-/Bederstr.",16.43
73,"Zürich, Waffenplatzstrasse",20.73
74,"Zürich, Wildbachstrasse",3.95


### 6. For each station, determine the buses that have stopped there at least once.

In [67]:
df_stops2 = pd.DataFrame()
for name in df['name']:
    path= f'/Users/andreacardinali/Documents/ZTBus_samples/{name}.csv'
    if os.path.exists(path):
        df_stops = pd.read_csv(path, na_values='-')
        df_stops['name'] = name
        df_stops2 = pd.concat([df_stops2, df_stops])

In [43]:
df_stops3 = pd.merge(df_stops2, df, on='name')

In [79]:
df_stops3.columns

Index(['name', 'time_iso', 'time_unix', 'electric_powerDemand',
       'gnss_altitude', 'gnss_course', 'gnss_latitude', 'gnss_longitude',
       'itcs_busRoute', 'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce', 'busNumber', 'startTime_iso',
       'startTime_unix', 'endTime_iso', 'endTime_unix', 'drivenDistance',
       'busRoute', 'energyConsumption', 'itcs_numberOfPassengers_mean',
       'itcs_numberOfPassengers_min', 'itcs_numberOfPassengers_max',
       'status_gridIsAvailable_mean', 'temperature_ambient_mean',
       'temp

In [102]:
prova = df_stops2.groupby(['itcs_stopName','name'], as_index=False).size()
prova

,itcs_stopName,name,size
0,"Zürich, Albisrank",B183_2021-07-26_03-53-27_2021-07-26_07-18-47,6
1,"Zürich, Albisrank",B183_2021-10-05_13-53-20_2021-10-05_18-14-53,8
2,"Zürich, Albisrank",B208_2021-05-27_04-09-18_2021-05-27_18-21-41,27
3,"Zürich, Albisrank",B208_2021-09-25_05-59-40_2021-09-25_18-15-31,22
4,"Zürich, Albisrank",B208_2022-11-30_04-47-53_2022-11-30_19-50-22,28
...,...,...,...
273,"Zürich, Wildbachstrasse",B183_2019-06-24_03-16-13_2019-06-24_18-54-06,37
274,"Zürich, Wildbachstrasse",B183_2020-07-19_03-04-41_2020-07-19_21-04-28,21
275,"Zürich, Wildbachstrasse",B208_2022-10-09_03-26-03_2022-10-09_22-56-13,26
276,"Zürich, Zwinglihaus",B208_2022-07-16_05-30-37_2022-07-16_18-19-51,19


### 7. For each station, determine the buses that have stopped there at least ten times.

In [101]:
prova[prova['size']>=10]

,itcs_stopName,name,size
2,"Zürich, Albisrank",B208_2021-05-27_04-09-18_2021-05-27_18-21-41,27
3,"Zürich, Albisrank",B208_2021-09-25_05-59-40_2021-09-25_18-15-31,22
4,"Zürich, Albisrank",B208_2022-11-30_04-47-53_2022-11-30_19-50-22,28
6,"Zürich, Albisriederplatz",B183_2020-07-19_03-04-41_2020-07-19_21-04-28,21
9,"Zürich, Albisriederplatz",B208_2021-05-27_04-09-18_2021-05-27_18-21-41,27
...,...,...,...
273,"Zürich, Wildbachstrasse",B183_2019-06-24_03-16-13_2019-06-24_18-54-06,37
274,"Zürich, Wildbachstrasse",B183_2020-07-19_03-04-41_2020-07-19_21-04-28,21
275,"Zürich, Wildbachstrasse",B208_2022-10-09_03-26-03_2022-10-09_22-56-13,26
276,"Zürich, Zwinglihaus",B208_2022-07-16_05-30-37_2022-07-16_18-19-51,19


### 8. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance. 

In [17]:
df2 = df.groupby(['busNumber','busRoute'])[['drivenDistance','energyConsumption']].sum()

In [18]:
df2.reset_index(inplace=True)

In [19]:
df2['ratio'] = df2['energyConsumption']/df2['drivenDistance']

In [20]:
df2

,busNumber,busRoute,drivenDistance,energyConsumption,ratio
0,183,31,2279952.00,1.387782e+10,6086.891785
1,183,32,2256540.10,1.393320e+10,6174.585331
2,183,33,29510481.41,1.761996e+11,5970.747507
3,183,46,18224657.94,1.026237e+11,5631.035959
4,183,72,23637238.45,1.394368e+11,5899.029643
5,183,83,51249813.70,2.995540e+11,5844.977154
6,183,N1,705122.71,4.217283e+09,5980.920966
7,183,N2,1390618.35,7.929098e+09,5701.850332
8,183,N4,766868.89,4.720068e+09,6154.986937
9,208,31,1046306.68,5.535865e+09,5290.862713


### 9. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [ ]:
groupmax = df2.groupby('busRoute')[['busNumber','ratio']].max()
groupmin = df2.groupby('busRoute')[['busNumber','ratio']].min()
df3 = pd.merge(groupmax, groupmin, on = 'busRoute')

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df3['difference'] = df3['ratio_x'] - df3['ratio_y']
df3.head()

,busRoute,busNumber_x,ratio_x,busNumber_y,ratio_y,difference
0,31.0,208,6086.891785,183,5290.862713,796.029072
1,32.0,208,6174.585331,183,5491.247671,683.337660
2,33.0,208,5970.747507,183,5639.244537,331.502969
3,46.0,208,5631.035959,183,5583.110707,47.925252
4,72.0,208,5899.029643,183,5410.318878,488.710765


### 10. Find the bus maximizing the difference computed in the previous point.

In [ ]:
df3.loc[df3['difference'].idxmax()]

busRoute         31.000000
busNumber_x     208.000000
ratio_x        6086.891785
busNumber_y     183.000000
ratio_y        5290.862713
difference      796.029072
Name: 0, dtype: float64

### 11. Extract the rows of the details such that the `gnss_altitude` differs from the value in the preceding row. Store also the difference in the variable `altitude_variation`.

### 12. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of `altitude_variation`.

### 13.  For each month of the year, compute the average ambient temperature

### 14. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

### 15. For each pair of stops that are consecutive in some trip, compute the average speed achieved when going from the first to the second stop.